In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage

In [ ]:
# .env dosyası içerisinde gerekli olan keyler alınır / Required keys are retrieved from the .env file
from dotenv import load_dotenv
_ = load_dotenv()

# Key bilgisi sağlamanın diğer yolu / Another way to provide the key

# import openai
# openai.api_key = "YOUR_OPENAI_API_KEY"

In [ ]:
# LLM Model
llm = OpenAI(model="gpt-4o-mini")

# Dökümanların yüklenmesi \\ Load Documents
data_dir = "data"
reader = SimpleDirectoryReader(data_dir)  # 'data' klasöründeki belgeleri okur
docs = reader.load_data()

# Embedding model
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002",vector_store_query_mode='default')

# Dokümanların indekslenmesi \\ Indexing to the documents
index = VectorStoreIndex.from_documents(docs, embed_model=embedding_model)
retriever = index.as_retriever(similarity_top_k=3)

In [7]:
def search_documents(query, threshold):
    # İlk aşama: Belge alma (Retriever ile) // First stage: Documentation (with Retriever)
    retrieved_docs = retriever.retrieve(query)  # İlk K doküman

    # Eğer belge yoksa, boş döndür // If there is no document, return blank
    if not retrieved_docs:
        return []

    # Retrieve edilen dokümanlar da eşik üstü kalan dokümanlar döndür // Return documents that are above the threshold among retrieved documents
    return [doc.text for doc in retrieved_docs if doc.score >= threshold]

In [10]:
def generate_response(query, threshold):
    relevant_docs = search_documents(query, threshold)
    context = "\n".join(relevant_docs)

    # Eğer herhangi bir döküman retrive edilmemişse sorguya herhangi bir ekleme yapılmaz 
    if not relevant_docs:
        prompt = f"Query: {query}" 
    # Eğer uygun bulunan döküman varsa prompt'a çekilen dökümanlar eklenir.
    else:
        prompt = f"Given the following documents, answer the query concisely.\n\nContext:\n{context}\n\nQuery: {query}" 
    print(prompt)
    messages = [
        ChatMessage(
            role="system", content=prompt
        ),
    ]

    response = llm.chat(messages)
    return f"Answer: {response.message.content}"

In [11]:
# Test
#query = "What are the latest AI trends?"
#query = "is cat animal"
query = "What is the AI?"
#query = "What is relationship between AI and Autonomous systems?"
#query = "What is AI history an relationship between AI and Autonomous systems?"
#query = "What is relationship among AI, Finance, Machine Learning and Autonomous systems?"
response = generate_response(query, threshold=0.75)
print(response)

Given the following documents, answer the query concisely.

Context:
Artificial intelligence (AI) refers to the development of computer systems that can perform tasks typically requiring human intelligence, such as problem-solving, decision-making, language understanding, and pattern recognition. AI is transforming industries by automating complex processes, improving efficiency, and enabling new innovations that were once thought impossible.
AI continues to evolve rapidly, with advancements in general AI, explainable AI, and AI-human collaboration. Researchers are exploring artificial general intelligence (AGI), where machines could possess human-like cognitive abilities. The future of AI could reshape society, making tasks easier while raising new ethical and economic considerations.
The concept of AI dates back to ancient myths and early philosophical discussions, but it became a formal field of study in the 1950s. Early AI models focused on rule-based systems, while modern AI lever

In [ ]:
# Vector DB içerisinde alakalı döküman yoksa
query = "Is cat animal"
response = generate_response(query, threshold=0.75)
print(response)

Query: Is cat animal
Answer: Yes, a cat is an animal. Specifically, it is a domesticated mammal belonging to the species Felis catus. Cats are known for their companionship with humans and are popular pets around the world.
